In [1]:
from google.cloud import storage
import re
import os

In [4]:
import pygrib
from datetime import datetime, timedelta
import urllib3
import ssl
import re
import numpy as np

In [77]:
client = storage.Client.create_anonymous_client()
bucket = client.get_bucket("high-resolution-rapid-refresh")
pattern = r"^hrrr\..*/conus/hrrr\.t(00|06|12)z\.wrfnatf(00|06|12)\.grib2$"
# pattern_idx = r"^hrrr\..*/conus/hrrr\.t(00|06|12)z\.wrfnatf(00|06|12)\.grib2.idx$"
# pattern = r"hrrr\.(2019|2020|2021|2022)(05|06|07|08|09)..\/conus\/hrrr\.t(00|06|12|18)z\.wrfnatf(00|06|12)\.grib2$"
# pattern = r"hrrr\.2019(05|06|07|08|09)..\/conus\/hrrr\.t(00|06|12|18)z\.wrfnatf(00|06|12)\.grib2$"
prefixes = ["hrrr.2019","hrrr.2020","hrrr.2021", "hrrr.2022"]

In [78]:
matching_blobs = []

In [79]:
blobs_2019=bucket.list_blobs(prefix="hrrr.2019")

In [80]:
import urllib.request

In [81]:
def get_blob_uri(blob):
    return "https://storage.googleapis.com/" + str(blob.id[:-(len(str(blob.generation)) + 1)])

In [82]:
# get start and end bytes
def get_bytes(file,variable):
    fileidx = urllib.request.urlopen(get_blob_uri(blob) + ".idx")
    #fileidx = urllib3.urlopen(blob.name + ".idx")
    try:
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        idxpage = urllib.request.urlopen(fileidx, context=ctx)
        #idxpage = urllib3.urlopen(fileidx, context=ctx)
    except:
        # idxpage = urllib3.urlopen(fileidx)
        idxpage = urllib.request.urlopen(fileidx, context=ctx)
    lines = idxpage.readlines()
    gcnt = 0
    for g in lines:
        expr = re.compile(variable)
        if expr.search(g):
            print('matched a variable', g)
            parts = g.split(':')
            rangestart = parts[1]
            parts = lines[gcnt+1].split(':')
            rangeend = int(parts[1])-1
            print ('range:', rangestart, rangeend)
            byte_range = str(rangestart) + '-' + str(rangeend)
            # # 2) When the byte range is discovered, use cURL to download.
            # os.system('curl -o %s --range %s %s' % (outfile, byte_range, pandofile))
        gcnt += 1
    return rangestart, rangeend

In [83]:
test_list = []
for blob in blobs_2019:
    if re.match(pattern, blob.name):
        print(blob)
        var = 'TMP'
        print(blob.name)
        test_list.append(blob)
        filename = blob.name[5:13] + blob.name[25:29] + blob.name[37:39] + ".grib2"
        print(filename)
        destination_path = "/Volumes/hrrr/native/" + filename + var
        print(destination_path)
        bytestart, byteend = get_bytes(blob.name, var)
        blob.download_to_filename(destination_path, bytestart,byteend) # download_to_filename(start=start_byte, end=end_byte)
        print(f"Downloaded {blob.name} to {destination_path}")

In [ ]:
matching_blobs = []

for prefix in prefixes:
    blobs = bucket.list_blobs(prefix=prefix)
    print("done blobs")
    matching_blobs.extend([blob for blob in blobs if re.match(pattern, blob.name)])
    print("done extend")

In [56]:
for blob in test:
    if re.match(pattern, blob.name):
        print(blob)


KeyboardInterrupt: 

In [ ]:
matching_blobs.extend([blob for blob in blobs if re.match(pattern, blob.name)])

In [45]:
matching_blobs = []

for prefix in prefixes:
    blobs = bucket.list_blobs(prefix=prefix)
    print("done blobs")
    matching_blobs.extend([blob for blob in blobs if re.match(pattern, blob.name)])
    print("done extend")



# for blob in blobs:
#     if re.match(pattern, blob.name):
#         filename = blob.name[5:13] + blob.name[25:29] + blob.name[37:39]
#         destination_path = os.path.join("/Volumes/hrrr/native/", filename)

#         blob.download_to_filename(destination_path)
#         print(f"Downloaded {blob.name} to {destination_path}")

done blobs


KeyboardInterrupt: 